In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [2]:
artical_info = pd.read_csv('Downloads/Article_Recommendation-201024-173805/Article_Recommendation/article_info.csv')

sample= pd.read_csv('Downloads/Article_Recommendation-201024-173805/Article_Recommendation/sample_submission.csv')

test=pd.read_csv('Downloads/Article_Recommendation-201024-173805/Article_Recommendation/test.csv')

train=pd.read_csv('Downloads/Article_Recommendation-201024-173805/Article_Recommendation/train.csv')

In [3]:
artical_info.head()

,article_id,website,title,content
0,1025,uxmovement,Comment concevoir une procédure pas à pas que ...,par anthony le 18/07/16 à 8h02 Si une nouvelle...
1,2328,endeavor,Ressources humaines? Seulement si vous optez p...,"«Ambassadeurs», «avocats», «porte-parole» d'un..."
2,2469,linkedin,Deux motions de vente différentes. . . .,J'ai passé pas mal de temps récemment avec des...
3,2590,googleblog,Apprentissage large et profond: mieux avec Ten...,"""Apprenez les règles comme un pro, afin de pou..."
4,697,infoq,Agile: manque de compétences en tests,"Fran O'Hara, directeur et consultant principal..."


In [96]:
sample.head()

(7243, 3)

In [5]:
test.head()

,user_id,article_id
0,1,2607
1,1,1445
2,1,911
3,1,857
4,1,2062


In [95]:
train.head()

,user_id,article_id,rating
0,1,456,1
1,1,2934,1
2,1,82,1
3,1,1365,1
4,1,221,1


In [7]:
import pandas as pd

# Load datasets
train = pd.read_csv("Downloads/Article_Recommendation-201024-173805/Article_Recommendation/train.csv")
article_info = pd.read_csv("Downloads/Article_Recommendation-201024-173805/Article_Recommendation/article_info.csv")
test = pd.read_csv("Downloads/Article_Recommendation-201024-173805/Article_Recommendation/test.csv")

# Check structure
print(train.head())
print(train.info())

# Check missing values
print(train.isnull().sum())


   user_id  article_id  rating
0        1         456       1
1        1        2934       1
2        1          82       1
3        1        1365       1
4        1         221       1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16731 entries, 0 to 16730
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   user_id     16731 non-null  int64
 1   article_id  16731 non-null  int64
 2   rating      16731 non-null  int64
dtypes: int64(3)
memory usage: 392.3 KB
None
user_id       0
article_id    0
rating        0
dtype: int64


In [8]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(train, test_size=0.2, random_state=42)


In [9]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
from scipy.sparse import csr_matrix

# Create user-item matrix
user_item_matrix = train.pivot(index='user_id', columns='article_id', values='rating').fillna(0)

# Convert to sparse matrix
matrix = csr_matrix(user_item_matrix.values)

# Fit Nearest Neighbors model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(matrix)

# Function to get recommendations
def get_user_based_recommendations(user_id, n_neighbors=5):
    user_idx = user_item_matrix.index.get_loc(user_id)
    distances, indices = model_knn.kneighbors(user_item_matrix.iloc[user_idx, :].values.reshape(1, -1), n_neighbors=n_neighbors)
    
    similar_users = user_item_matrix.index[indices.flatten()]
    return similar_users

print(get_user_based_recommendations(user_id=123))  # Example user


Index([123, 958, 978, 41, 574], dtype='int64', name='user_id')


In [10]:
item_similarity = cosine_similarity(user_item_matrix.T)
item_sim_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns).fillna(0)


In [11]:
def get_item_based_recommendations(article_id, n=5):
    if article_id not in item_sim_df.index:
        print(f"Article ID {article_id} not found in similarity matrix. Returning most popular articles instead.")
        return train.groupby("article_id")["rating"].mean().sort_values(ascending=False).head(n)
    
    return item_sim_df[article_id].sort_values(ascending=False).iloc[1:n+1]

print(get_item_based_recommendations(article_id=101))


Article ID 101 not found in similarity matrix. Returning most popular articles instead.
article_id
1628    5.0
2282    5.0
479     5.0
821     5.0
2190    5.0
Name: rating, dtype: float64


In [12]:
#Assign X as the original ratings dataframe
X = train.copy()

#Split into training and test datasets
X_train, X_test = train_test_split(X, test_size = 0.25, random_state=42)

In [13]:
#Function that computes the root mean squared error (or RMSE)
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [14]:
#Define the baseline model to always return average of all available ratings
def baseline(user_id, artical_id):
    return X_train['rating'].mean()

In [15]:
#Function to compute the RMSE score obtained on the test set by a model
def rmse_score(model):
    
    #Construct a list of user-movie tuples from the test dataset
    id_pairs = zip(X_test['user_id'], X_test['article_id'])
    
    #Predict the rating for every user-movie tuple
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    
    #Extract the actual ratings given by the users in the test data
    y_true = np.array(X_test['rating'])
    
    #Return the final RMSE score
    return rmse(y_true, y_pred)

In [16]:
rmse_score(baseline)

0.9683927490470934

In [17]:
#Build the ratings matrix using pivot_table function
r_matrix = X_train.pivot_table(values='rating', index='user_id', columns='article_id')

r_matrix.head()

article_id,1,3,4,6,7,8,9,10,11,12,...,2963,2964,2965,2966,2968,2969,2970,2974,2975,2976
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#User Based Collaborative Filter using Mean Ratings
def cf_user_mean(user_id, article_id):
    
    #Check if movie exists in r_matrix
    if article_id in r_matrix:
        
        #Compute the mean of all the ratings given to the movie
        mean_rating = r_matrix[article_id].mean()
    
    else:
        #Default to average rating from the train set
        mean_rating = X_train['rating'].mean()
    
    return mean_rating

In [19]:
#Compute RMSE for the Mean model
rmse_score(cf_user_mean)

1.0420497615166175

In [20]:
#Compute the Pearson Correlation using the ratings matrix with corr function from Pandas
pearson_corr = r_matrix.T.corr()

In [21]:
#Convert into pandas dataframe 
pearson_corr = pd.DataFrame(pearson_corr, index=r_matrix.index, columns=r_matrix.index)

pearson_corr.head(10)

user_id,1,2,3,5,7,8,9,10,11,12,...,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#Fill all the missing correlations with 0
pearson_cor = pearson_corr.fillna(0)

In [23]:
#User Based Collaborative Filter using Weighted Mean Ratings
def cf_user_wmean(user_id, article_id):
    
    #Check if movie_id exists in r_matrix
    if article_id in r_matrix:
        
        #Mean rating for active user
        ra = r_matrix.loc[user_id].mean()

        #Get the similarity scores for the user in question with every other user
        sim_scores = pearson_corr[user_id].sort_values(ascending = False)
        
        # Keep similarity scores for users with positive correlation with active user
        sim_scores_pos = sim_scores[sim_scores > 0]
        
        #Get the user ratings for the movie in question
        m_sample = r_matrix[article_id][sim_scores_pos.index]
        
        #Extract the indices containing NaN in the m_ratings series (Users who have not rated the target movie)
        idx = m_sample[m_sample.isnull()].index
        
        #Drop the NaN values from the m_ratings Series
        m_sample = m_sample.dropna()
        
        # If there are no ratings from similar users we cannot use this method so we predict just 
        # the average rating of the movie else we use the prediction formula
        if len(m_sample) == 0:
            #Default to average rating in the absence of ratings by similar users
            wmean_rating = r_matrix[article_id].mean()
        else:   
            #Drop the corresponding correlation scores from the sim_scores series
            sim_scores_pos = sim_scores_pos.drop(idx)
            
            #Subtract average rating of each user from the rating (rbp - mean(rb))
            m_sample = m_sample - r_matrix.loc[m_sample.index].mean(axis = 1)
            
            #Compute the final weighted mean using np.dot which is nothing but the product divided by sum of weights
            wmean_rating = ra + (np.dot(sim_scores_pos, m_sample)/ sim_scores_pos.sum())
   
    else:
        #Default to average rating in the absence of any information on the movie in train set
        wmean_rating = X_train['rating'].mean()
    
    return wmean_rating

In [24]:
rmse_score(cf_user_wmean)

1.0950466136265755

In [25]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [26]:
sample.columns

Index(['user_id', 'article_id', 'rating'], dtype='object')

In [27]:
#Assign X as the original ratings dataframe
X = sample.copy()

#Split into training and test datasets
X_train, X_test = train_test_split(X, test_size = 0.25, random_state=42)

In [28]:
#Function that computes the root mean squared error (or RMSE)
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [29]:
#Importing functions to be used in this notebook from Surprise Package
from surprise import Dataset, Reader
from surprise.model_selection import GridSearchCV
from surprise.prediction_algorithms import KNNWithMeans

In [30]:
#Reader object to import ratings from X_train
reader = Reader(rating_scale=(1, 5))

#Storing Data in surprise format from X_train
data = Dataset.load_from_df(X_train[['user_id','article_id','rating']], reader)

In [31]:
#Defining the parameter grid with k as the neighbourhood size & trying 2 similarity measures KNNwithMeans
#& 5 folds
param_grid = {"k":list(range(1,50,5)),
              "sim_options":{"name":["cosine","pearson"]}}

#KNNWithMeans by default does user based collaborative filtering and here we are trying to find the best set of 
#parameters
gs = GridSearchCV(KNNWithMeans, 
                  param_grid, 
                  measures=['rmse'], 
                  cv=5, 
                  n_jobs = -1)

#We fit the grid search on data to find out the best score
gs.fit(data)

#Printing the best score
print(gs.best_score['rmse'])

#Printing the best set of parameters
print(gs.best_params['rmse'])

0.0
{'k': 1, 'sim_options': {'name': 'cosine', 'user_based': True}}


In [32]:
#Defining similarity measure as per the best parameters
sim_options = {'name': 'pearson'}

#Fitting the model on train data
model = KNNWithMeans(k = 46, sim_options = sim_options)

#Build full trainset will essentially fit the knnwithmeans on the complete train set instead of a part of it
#like we do in cross validation
model.fit(data.build_full_trainset())

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [33]:
#id pairs for test set
id_pairs = zip(X_test['user_id'], X_test['article_id'])

#Making predictions for test set using predict method from Surprise
[model.predict(uid = user, iid = article_id) for (user, article_id) in id_pairs]

[Prediction(uid=72, iid=2871, r_ui=None, est=1, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=612, iid=937, r_ui=None, est=1, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=556, iid=1861, r_ui=None, est=1, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=232, iid=1663, r_ui=None, est=1, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=470, iid=215, r_ui=None, est=1, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=719, iid=2108, r_ui=None, est=1, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=259, iid=1106, r_ui=None, est=1, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=231, iid=2934, r_ui=None, est=1, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=1020, iid=2392, r_ui=None, est=1, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=579, iid=1414, r_ui=None, est=1, details={'actual_k': 0, 'wa

In [34]:
#id pairs for test set
id_pairs = zip(X_test['user_id'], X_test['article_id'])

#Making predictions for test set using predict method from Surprise
y_pred = [model.predict(uid = user, iid = article_id)[3] for (user, article_id) in id_pairs]

#Actual rating values for test set
y_true = X_test['rating']

# Checking performance on test set
rmse(y_true, y_pred)

0.0

In [35]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [36]:
sample.columns

Index(['user_id', 'article_id', 'rating'], dtype='object')

In [37]:
#Assign X as the original ratings dataframe
X = sample.copy()

#Split into training and test datasets
X_train, X_test = train_test_split(X, test_size = 0.25, random_state=42)

In [38]:
#Function that computes the root mean squared error (or RMSE)
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [50]:
#Define the baseline model to always return average of all available ratings
def baseline(user_id, article_id):
    return X_train['rating'].mean()

In [51]:
#Function to compute the RMSE score obtained on the test set by a model
def rmse_score(model):
    
    #Construct a list of user-movie tuples from the test dataset
    id_pairs = zip(X_test['user_id'], X_test['article_id'])
    
    #Predict the rating for every user-movie tuple
    y_pred = np.array([model(user, article_id) for (user, article_id) in id_pairs])
    
    #Extract the actual ratings given by the users in the test data
    y_true = np.array(X_test['rating'])
    
    #Return the final RMSE score
    return rmse(y_true, y_pred)

In [52]:
rmse_score(baseline)

0.0

In [107]:
#Build the ratings matrix using pivot_table function
r_matrix = X_train.pivot_table(values='rating', index='user_id', columns='article_id')

r_matrix.head()

article_id,1,3,4,6,7,8,11,12,13,14,...,2962,2963,2964,2966,2968,2969,2970,2974,2975,2976
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
import numpy as np

def cf_item_mean(user_id, article_id):
    if article_id not in item_sim_df.index:
        # If item is missing, return the user's mean rating as a fallback
        return r_matrix.loc[user_id].mean() if user_id in r_matrix.index else np.nan
    
    # Find similar articles and their similarity scores
    similar_articles = item_sim_df[article_id].drop(article_id).sort_values(ascending=False)
    
    # Get user's ratings for those articles
    rated_articles = r_matrix.loc[user_id, similar_articles.index].dropna()
    
    if rated_articles.empty:
        return r_matrix.loc[user_id].mean()  # Fallback to user's mean rating
    
    # Compute weighted average of similar articles' ratings
    weights = similar_articles.loc[rated_articles.index]
    prediction = np.dot(rated_articles, weights) / weights.sum()
    
    return prediction

# Example usage:
predicted_rating = cf_item_mean(user_id=123, article_id=101)
print(f"Predicted Rating: {predicted_rating}")


Predicted Rating: 1.0


In [56]:
#Create a dummy ratings matrix with all null values imputed to 0
r_matrix_dummy = r_matrix.copy().fillna(0)

In [57]:
#Compute the cosine similarity matrix using the dummy ratings matrix
cosine_sim = cosine_similarity(r_matrix_dummy.T, r_matrix_dummy.T)

#Convert into pandas dataframe 
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.columns, columns=r_matrix.columns)

cosine_sim.head(5)

article_id,1,3,4,6,7,8,11,12,13,14,...,2962,2963,2964,2966,2968,2969,2970,2974,2975,2976
article_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25
6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [61]:
#Importing functions to be used in this notebook from Surprise Package
from surprise import Dataset, Reader
from surprise.model_selection import GridSearchCV
from surprise.prediction_algorithms import KNNWithMeans

In [62]:
#Reader object to import ratings from X_train
reader = Reader(rating_scale=(1, 5))

#Storing Data in surprise format from X_train
data = Dataset.load_from_df(X_train[['user_id','article_id','rating']], reader)

In [63]:
#Defining the parameter grid with k as the neighbourhood size & trying 2 similarity measures KNNwithMeans
#& 5 folds, we also use user_based as True and false to try both user based and item based collaborative filtering
#and check which performs better
param_grid = {"k":list(range(1,50,5)),
              "sim_options":{"name":["cosine","pearson"],'user_based': [True,False]}}

#Trying to find the best set of hyperparameters using Grid Search
gs = GridSearchCV(KNNWithMeans, 
                  param_grid, 
                  measures=['rmse'], 
                  cv=5, 
                  n_jobs = -1)

#We fit the grid search on data to find out the best score
gs.fit(data)

#Printing the best score
print(gs.best_score['rmse'])

#Printing the best set of parameters
print(gs.best_params['rmse'])

0.0
{'k': 1, 'sim_options': {'name': 'cosine', 'user_based': True}}


In [64]:
#Defining similarity measure as per the best parameters
sim_options = {'name': 'cosine', 'user_based': False}

#Fitting the model on train data
model = KNNWithMeans(k = 46, sim_options = sim_options)

#Build full trainset will essentially fits the knnwithmeans on the complete train set instead of a part of it
#like we do in cross validation
model.fit(data.build_full_trainset())

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [65]:
#id pairs for test set
id_pairs = zip(X_test['user_id'], X_test['article_id'])

#Making predictions for test set using predict method from Surprise
y_pred = [model.predict(uid = user, iid = article_id)[3] for (user, article_id) in id_pairs]

#Actual rating values for test set
y_true = X_test['rating']

# Checking performance on test set
rmse(y_true, y_pred)

0.0

In [80]:
import pandas as pd

# Load datasets
train = pd.read_csv("Downloads/Article_Recommendation-201024-173805/Article_Recommendation/train.csv")
article_info = pd.read_csv("Downloads/Article_Recommendation-201024-173805/Article_Recommendation/article_info.csv")
test = pd.read_csv("Downloads/Article_Recommendation-201024-173805/Article_Recommendation/test.csv")

# Check structure
print(train.head())
print(train.info())

# Check missing values
print(train.isnull().sum())


   user_id  article_id  rating
0        1         456       1
1        1        2934       1
2        1          82       1
3        1        1365       1
4        1         221       1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16731 entries, 0 to 16730
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   user_id     16731 non-null  int64
 1   article_id  16731 non-null  int64
 2   rating      16731 non-null  int64
dtypes: int64(3)
memory usage: 392.3 KB
None
user_id       0
article_id    0
rating        0
dtype: int64


In [81]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(train, test_size=0.2, random_state=42)


In [82]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
from scipy.sparse import csr_matrix

# Create user-item matrix
user_item_matrix = train.pivot(index='user_id', columns='article_id', values='rating').fillna(0)

# Convert to sparse matrix
matrix = csr_matrix(user_item_matrix.values)

# Fit Nearest Neighbors model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(matrix)

# Function to get recommendations
def get_user_based_recommendations(user_id, n_neighbors=5):
    user_idx = user_item_matrix.index.get_loc(user_id)
    distances, indices = model_knn.kneighbors(user_item_matrix.iloc[user_idx, :].values.reshape(1, -1), n_neighbors=n_neighbors)
    
    similar_users = user_item_matrix.index[indices.flatten()]
    return similar_users

print(get_user_based_recommendations(user_id=123))  # Example user


Index([123, 958, 978, 41, 574], dtype='int64', name='user_id')


In [97]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute item similarity
item_similarity = cosine_similarity(user_item_matrix.T)

# Convert to DataFrame
item_sim_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

# Function to get similar articles
def get_item_based_recommendations(article_id, n=5):
    return item_sim_df[article_id].sort_values(ascending=False).iloc[1:n+1]

print(get_item_based_recommendations(article_id=102))  # Example article


article_id
361     0.948683
1304    0.948683
1052    0.948683
2109    0.948683
1506    0.948683
Name: 102, dtype: float64


In [98]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

# Prepare data for Surprise library
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(train[['user_id', 'article_id', 'rating']], reader)

# Train SVD model
svd = SVD()
cross_validate(svd, data, cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9311  0.9424  0.9416  0.9387  0.9367  0.9381  0.0041  
MAE (testset)     0.6150  0.6188  0.6178  0.6205  0.6165  0.6177  0.0019  
Fit time          0.34    0.25    0.27    0.24    0.25    0.27    0.03    
Test time         0.03    0.03    0.03    0.03    0.04    0.03    0.00    


{'test_rmse': array([0.93107965, 0.94240901, 0.94162185, 0.93871276, 0.93666657]),
 'test_mae': array([0.61497424, 0.61881795, 0.61782915, 0.62050372, 0.6164916 ]),
 'fit_time': (0.33706140518188477,
  0.25258398056030273,
  0.26662302017211914,
  0.24246716499328613,
  0.2479724884033203),
 'test_time': (0.031273841857910156,
  0.031239748001098633,
  0.03307986259460449,
  0.03121185302734375,
  0.038877010345458984)}

In [99]:
from surprise import accuracy
from surprise.model_selection import train_test_split

# Train-test split
trainset, testset = train_test_split(data, test_size=0.2)
svd.fit(trainset)

# Predict on test set
predictions = svd.test(testset)

# Compute RMSE
rmse = accuracy.rmse(predictions)
print("SVD RMSE:", rmse)


RMSE: 0.9421
SVD RMSE: 0.9421229442580193


In [100]:
# Make predictions for test.csv
test['rating'] = test.apply(lambda row: svd.predict(row['user_id'], row['article_id']).est, axis=1)

# Save submission
test[['user_id', 'article_id', 'rating']].to_csv("submission.csv", index=False)


In [101]:
submission = pd.read_csv("Downloads/Article_Recommendation-201024-173805/Article_Recommendation/sample_submission.csv")
print(submission.head())
print(submission.info())  # Ensure all columns exist and no missing values


   user_id  article_id  rating
0        1        2607       1
1        1        1445       1
2        1         911       1
3        1         857       1
4        1        2062       1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7243 entries, 0 to 7242
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   user_id     7243 non-null   int64
 1   article_id  7243 non-null   int64
 2   rating      7243 non-null   int64
dtypes: int64(3)
memory usage: 169.9 KB
None


In [102]:
from surprise import accuracy
from surprise.model_selection import train_test_split

# Split data
trainset, testset = train_test_split(data, test_size=0.2)

# Train SVD model
svd.fit(trainset)

# Make predictions
predictions = svd.test(testset)

# Compute RMSE
rmse = accuracy.rmse(predictions)
print(f"Validation RMSE: {rmse}")


RMSE: 0.9635
Validation RMSE: 0.96346958549229


In [103]:
from surprise.model_selection import GridSearchCV

# Define hyperparameter grid
param_grid = {
    'n_factors': [50, 100, 150],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.05, 0.1]
}

# Perform GridSearch
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(data)

# Best RMSE score
print(f"Best RMSE: {gs.best_score['rmse']}")
print(f"Best params: {gs.best_params['rmse']}")

# Train best model
best_svd = gs.best_estimator['rmse']
best_svd.fit(trainset)


Best RMSE: 0.9374809681703411
Best params: {'n_factors': 50, 'lr_all': 0.005, 'reg_all': 0.1}
